In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

print("OpenAI API Loaded:", bool(os.getenv("OPENAI_API_KEY")))

OpenAI API Loaded: True


In [13]:
from openai import OpenAI

client = OpenAI()

# 🏀 Basketball Scouting Prompt Distillation (Notebook 02)

**Goal:**  
Build the foundation for a distilled model that turns **raw scouting notes**  
(stats, playstyle blurbs, subjective comments) into a **clean, structured JSON scouting report**.

This notebook focuses on:

1. Defining the output **scouting report JSON schema**
2. Creating a **small hand-written dataset** (our *gold examples*)
3. Saving them as `.jsonl` in a format Tinker expects
4. Preparing for the next notebook where we:
   - Use a **teacher model** to expand the dataset
   - Train a **small LoRA student** using Tinker’s distillation pipeline

By the end of Notebook 02, we’ll have:
- A real dataset
- A clear schema  
- A repeatable pattern for adding more examples  
- A working foundation for Tinker training

In [1]:
from pathlib import Path
from dataclasses import dataclass
import json

# Locate project root: assume this notebook lives in /notebooks
PROJECT_ROOT = Path.cwd().resolve().parents[0]

DATA_DIR = PROJECT_ROOT / "data" / "basketball_distillation"
DATA_DIR.mkdir(parents=True, exist_ok=True)

DATA_FILE = DATA_DIR / "scouting_examples.jsonl"

PROJECT_ROOT, DATA_DIR, DATA_FILE

(WindowsPath('C:/Users/user/Desktop/tinker-hello-world'),
 WindowsPath('C:/Users/user/Desktop/tinker-hello-world/data/basketball_distillation'),
 WindowsPath('C:/Users/user/Desktop/tinker-hello-world/data/basketball_distillation/scouting_examples.jsonl'))

## 📐 Scouting Report JSON Schema

Our distilled model will learn to output a **structured scouting report**.

Here is the schema we will standardize on:

```json
{
  "player_name": "",
  "team": "",
  "birthdate": "",
  "position": "",
  "year": "",
  "conference": "",
  "archetype": "",
  "offense_summary": "",
  "defense_summary": "",
  "intangibles": "",
  "projection": ""
}

---

## 🧠 Why We Define the Schema First

Before generating data or training a model, we need **absolute clarity** on what the model
is supposed to produce. This is the single most important design choice for
prompt distillation:

> **A distilled model can only mimic the behavior we teach it.  
> A clean, consistent schema → clean, consistent model outputs.**

This JSON structure will drive everything downstream:

- Teacher prompts  
- Synthetic example generation  
- LoRA training  
- Evaluation  
- Integration into larger analytics tools  

By locking the schema now, we avoid model drift and build a system that can plug into
our future college basketball dashboards, player similarity search, and scouting automations.

---

In [2]:
from dataclasses import dataclass

@dataclass
class ScoutingExample:
    # raw input we’d feed the teacher/student model
    query: str       # stats + notes, free-text
    # desired output (the structured scouting JSON as a string)
    response: str    # JSON string of scouting report


def to_jsonl_line(example: ScoutingExample) -> str:
    """
    Convert an example into a JSONL line.

    Tinker-style format:
    {
      "query": "...raw notes...",
      "response": "...JSON scouting report string..."
    }
    """
    return json.dumps(
        {"query": example.query, "response": example.response},
        ensure_ascii=False,
    )

## 🧱 Creating a Data Structure for Training Examples

We need a simple, explicit way to store our examples in memory before saving them to disk.

Each example contains two parts:

1. **query** → the raw scouting notes (messy text)
2. **response** → the structured JSON report we want the student model to learn

The `ScoutingExample` dataclass gives us a clean, typed object for each example.

We also define a helper `to_jsonl_line()`:

- Converts the example into a single JSON object  
- Ensures the final file matches Tinker’s required format (`{"query": ..., "response": ...}`)  
- Allows us to save multiple examples in `.jsonl` format (one object per line)

This format is perfect for dataset expansion and LoRA training.

---

## ✍️ Creating Our First Gold Example (On-Ball Creator Guard)

We now begin constructing **hand-written gold examples**.

Why hand-written?

- They establish the **gold standard** for what “good” output looks like  
- They shape the behavior the teacher model will generalize  
- They capture the exact tone, detail level, and structure we want  
- They help the student model converge with fewer training steps

This first example represents:

**Archetype:** On-ball creator guard  
**Context:** ACC guard with strong PnR usage, bursty driving, streaky shooting, emotional leadership  

We include:

- Biographical info  
- Stats from multiple games  
- Subjective notes (role, habits, tendencies)  
- A fully structured JSON response using our schema  

This becomes our template for all future examples.

In [3]:
raw_examples = []

guard_example = ScoutingExample(
    query=(
        "Player: Jamal Rivers\n"
        "School: NC State\n"
        "Conference: ACC\n"
        "Position: 6'4 guard (combo)\n"
        "Year: Sophomore\n"
        "Birthdate: 2005-03-14\n\n"
        "Recent games:\n"
        "- vs Duke: 24 pts, 5 ast, 3 reb, 3 TO, 9/17 FG, 3/7 3PT\n"
        "- vs UNC: 18 pts, 7 ast, 2 reb, 2 TO, 6/13 FG, 2/5 3PT\n"
        "- vs Wake: 16 pts, 8 ast, 4 reb, 1 TO, 5/12 FG\n\n"
        "Notes:\n"
        "- Primary on-ball creator for long stretches.\n"
        "- Good burst, consistently gets two feet in the paint.\n"
        "- Pull-up three is improving but still streaky.\n"
        "- Can get loose with the ball vs pressure (careless TOs).\n"
        "- Competes defensively but dies on some screens.\n"
        "- Vocal, clear emotional leader for this group."
    ),
    response=json.dumps(
        {
            "player_name": "Jamal Rivers",
            "team": "NC State",
            "birthdate": "2005-03-14",
            "position": "Guard (combo)",
            "year": "Sophomore",
            "conference": "ACC",
            "archetype": "On-ball creator guard",
            "offense_summary": (
                "Primary creator who lives on the ball. Creates paint touches with "
                "burst and change of pace, comfortable in pick-and-roll. Pull-up "
                "three is trending up but remains streaky, which affects consistency. "
                "Turnovers mostly come from loose handle against pressure and trying "
                "to force hero passes, not lack of vision."
            ),
            "defense_summary": (
                "Competitive at the point of attack and uses length to bother smaller "
                "guards. Engagement is generally good, but he can die on screens and "
                "lose focus off the ball. Overall more disruptive than liability at "
                "this level when locked in."
            ),
            "intangibles": (
                "Vocal leader who organizes teammates and embraces big moments. "
                "Plays with edge and confidence. Needs continued growth in emotional "
                "control after missed calls and in late-game decision-making."
            ),
            "projection": (
                "Projects as a high-major starting guard with real on-ball creation "
                "value and a path to pro rotation minutes as a secondary creator if "
                "shooting and decision-making continue to improve."
            ),
        },
        ensure_ascii=False,
    ),
)

raw_examples.append(guard_example)
len(raw_examples)

1

In [4]:
print("QUERY:\n", raw_examples[0].query[:500], "\n")
print("RESPONSE JSON:\n", raw_examples[0].response[:500])

QUERY:
 Player: Jamal Rivers
School: NC State
Conference: ACC
Position: 6'4 guard (combo)
Year: Sophomore
Birthdate: 2005-03-14

Recent games:
- vs Duke: 24 pts, 5 ast, 3 reb, 3 TO, 9/17 FG, 3/7 3PT
- vs UNC: 18 pts, 7 ast, 2 reb, 2 TO, 6/13 FG, 2/5 3PT
- vs Wake: 16 pts, 8 ast, 4 reb, 1 TO, 5/12 FG

Notes:
- Primary on-ball creator for long stretches.
- Good burst, consistently gets two feet in the paint.
- Pull-up three is improving but still streaky.
- Can get loose with the ball vs pressure (careles 

RESPONSE JSON:
 {"player_name": "Jamal Rivers", "team": "NC State", "birthdate": "2005-03-14", "position": "Guard (combo)", "year": "Sophomore", "conference": "ACC", "archetype": "On-ball creator guard", "offense_summary": "Primary creator who lives on the ball. Creates paint touches with burst and change of pace, comfortable in pick-and-roll. Pull-up three is trending up but remains streaky, which affects consistency. Turnovers mostly come from loose handle against pressure and tryin

---

## 🏀 Gold Example #2 — Rim-Running Energy Big

The second example adds archetype diversity to our dataset.

Why does this matter?

A good teacher model needs to see **range**:

- Different positions  
- Different stat profiles  
- Different physical tools  
- Different scouting language  
- Different strengths/weaknesses patterns  

This example teaches the teacher/student:

- How to summarize rim-pressure offense  
- How to express limited shooting + role acceptance  
- How to phrase defensive strengths (rim protection, motor, rotation discipline)  
- How to describe physicality, motor, and activity-based value

These differences matter when generating synthetic examples later.

---

In [5]:
big_example = ScoutingExample(
    query=(
        "Player: Marcus Lane\n"
        "School: Arkansas\n"
        "Conference: SEC\n"
        "Position: 6'8 forward (small-ball 5)\n"
        "Year: Junior\n"
        "Birthdate: 2004-01-22\n\n"
        "Recent games:\n"
        "- vs Kentucky: 10 pts, 11 reb (5 OR), 2 blk\n"
        "- vs Tennessee: 8 pts, 9 reb, 3 blk\n"
        "- vs LSU: 14 pts, 8 reb, 1 blk, 6/7 FG (all at rim)\n\n"
        "Notes:\n"
        "- Very high motor, sprints the floor every possession.\n"
        "- Limited touch outside 10 feet, mostly finishes at the rim.\n"
        "- Good timing as weak-side rim protector.\n"
        "- Can switch onto some guards in a pinch.\n"
        "- FT% still under 60%; teams intentionally foul late.\n"
    ),
    response=json.dumps(
        {
            "player_name": "Marcus Lane",
            "team": "Arkansas",
            "birthdate": "2004-01-22",
            "position": "Forward / Small-ball 5",
            "year": "Junior",
            "conference": "SEC",
            "archetype": "Rim-running energy big",
            "offense_summary": (
                "Plays entirely inside the arc as a vertical spacer and rim runner. "
                "Generates efficient offense through cuts, rolls and offensive rebounds. "
                "Finishes well at the rim but offers little touch beyond 10 feet. "
                "Free-throw shooting remains a limiting issue in high-leverage minutes."
            ),
            "defense_summary": (
                "High-motor defender who provides timely weak-side rim protection. "
                "Can survive in switches for brief stretches against smaller guards. "
                "Defensive value tied to consistent activity and discipline, but foul "
                "rate can spike when late on rotations."
            ),
            "intangibles": (
                "Relentless energy big who boosts team pace and physicality. "
                "Accepts role and plays within himself. Needs improved communication "
                "on back-line coverages and composure at the free-throw line."
            ),
            "projection": (
                "Projects as a rotational high-major energy big with realistic "
                "pro upside if touch and free-throw shooting improve."
            ),
        },
        ensure_ascii=False,
    ),
)

raw_examples.append(big_example)
len(raw_examples)

2

---

## 🎯 Gold Example #3 — Prototypical 3&D Wing

This example covers a cornerstone archetype in modern scouting:

**3&D wings** are:

- High-leverage  
- High-value  
- Highly projectable for pro contexts

This example teaches our model:

- How to describe shooting gravity  
- What balanced, disciplined defense looks like  
- How to talk about role clarity  
- How to encode high-floor players in JSON format

By now, we have three strong, distinct archetypes:
1. On-ball creator  
2. Rim-running big  
3. 3&D wing  

This forms a great seed dataset for the next stage: synthetic expansion.

---

In [6]:
wing_example = ScoutingExample(
    query=(
        "Player: Devin Clark\n"
        "School: Colorado\n"
        "Conference: Pac-12\n"
        "Position: 6'6 wing\n"
        "Year: Senior\n"
        "Birthdate: 2002-11-03\n\n"
        "Recent games:\n"
        "- vs Arizona: 12 pts, 4/8 3PT, 6 reb, 2 stl\n"
        "- vs USC: 9 pts, 3/6 3PT, 5 reb, 1 blk\n"
        "- vs Oregon: 14 pts, 4/9 3PT, 7 reb, 3 ast\n\n"
        "Notes:\n"
        "- Reliable spot-up shooter with deep range.\n"
        "- Good defender, strong frame, disciplined on closeouts.\n"
        "- Limited self-creation off the bounce.\n"
        "- Makes simple reads, doesn't force plays.\n"
        "- Strong rebounder for position.\n"
    ),
    response=json.dumps(
        {
            "player_name": "Devin Clark",
            "team": "Colorado",
            "birthdate": "2002-11-03",
            "position": "Wing",
            "year": "Senior",
            "conference": "Pac-12",
            "archetype": "3&D wing",
            "offense_summary": (
                "High-level spot-up shooter with confident mechanics and deep range. "
                "Attacks closeouts in straight lines but has limited creation beyond "
                "one or two dribbles. Makes simple, correct reads and rarely forces shots."
            ),
            "defense_summary": (
                "Strong positional defender with a sturdy frame and disciplined footwork. "
                "Good technique on closeouts, stays balanced, and uses length well. "
                "Rebounds effectively for a wing and provides occasional weak-side contests."
            ),
            "intangibles": (
                "Reliable, low-maintenance rotation wing. Consistently competes, stays in "
                "his lane offensively, and shows mature decision-making. High floor due to "
                "role clarity and shooting consistency."
            ),
            "projection": (
                "Projects as a plug-and-play 3&D wing at the high-major level with "
                "professional viability due to shooting volume, defensive reliability, "
                "and physical maturity."
            ),
        },
        ensure_ascii=False,
    ),
)

raw_examples.append(wing_example)
len(raw_examples)

3

In [7]:
def save_examples_to_jsonl(examples, path: Path):
    with path.open("w", encoding="utf-8") as f:
        for ex in examples:
            f.write(to_jsonl_line(ex) + "\n")

save_examples_to_jsonl(raw_examples, DATA_FILE)

print("Saved examples to:", DATA_FILE)
print("Total examples saved:", len(raw_examples))

Saved examples to: C:\Users\user\Desktop\tinker-hello-world\data\basketball_distillation\scouting_examples.jsonl
Total examples saved: 3


In [8]:
preview_text = DATA_FILE.read_text(encoding="utf-8")
print(preview_text[:1200])

{"query": "Player: Jamal Rivers\nSchool: NC State\nConference: ACC\nPosition: 6'4 guard (combo)\nYear: Sophomore\nBirthdate: 2005-03-14\n\nRecent games:\n- vs Duke: 24 pts, 5 ast, 3 reb, 3 TO, 9/17 FG, 3/7 3PT\n- vs UNC: 18 pts, 7 ast, 2 reb, 2 TO, 6/13 FG, 2/5 3PT\n- vs Wake: 16 pts, 8 ast, 4 reb, 1 TO, 5/12 FG\n\nNotes:\n- Primary on-ball creator for long stretches.\n- Good burst, consistently gets two feet in the paint.\n- Pull-up three is improving but still streaky.\n- Can get loose with the ball vs pressure (careless TOs).\n- Competes defensively but dies on some screens.\n- Vocal, clear emotional leader for this group.", "response": "{\"player_name\": \"Jamal Rivers\", \"team\": \"NC State\", \"birthdate\": \"2005-03-14\", \"position\": \"Guard (combo)\", \"year\": \"Sophomore\", \"conference\": \"ACC\", \"archetype\": \"On-ball creator guard\", \"offense_summary\": \"Primary creator who lives on the ball. Creates paint touches with burst and change of pace, comfortable in pick-

In [9]:
loaded = []
with DATA_FILE.open("r", encoding="utf-8") as f:
    for line in f:
        loaded.append(json.loads(line))

print("Loaded examples:", len(loaded))
print("Keys:", loaded[0].keys())
print("\nFirst example (truncated):\n", loaded[0]["response"][:300])

Loaded examples: 3
Keys: dict_keys(['query', 'response'])

First example (truncated):
 {"player_name": "Jamal Rivers", "team": "NC State", "birthdate": "2005-03-14", "position": "Guard (combo)", "year": "Sophomore", "conference": "ACC", "archetype": "On-ball creator guard", "offense_summary": "Primary creator who lives on the ball. Creates paint touches with burst and change of pace, 


---

# 🧠 Designing the Teacher Prompt

The teacher prompt is the *blueprint* for the behavior we want to distill
into a smaller, cheaper specialist model.

This is the most important part of the pipeline.

The teacher model will:

- Read messy scouting input (stats + notes)
- Follow strict instructions
- Generate a **structured JSON scouting report**
- Match our schema *exactly*
- Use consistent scouting language
- Avoid hallucinating or inventing facts
- Only use information present in the notes or stats

We will later use this teacher prompt to auto-generate a large synthetic dataset.

Every distilled student model will learn from this pattern,
so we invest time now to make it perfect.

---

In [10]:
TEACHER_PROMPT_TEMPLATE = """
You are an expert basketball scout and analytics assistant.

Your task is to convert RAW_SCOUTING_NOTES into a structured JSON scouting report.
Follow these rules exactly:

1. **Always output valid JSON.**
2. **Use the exact schema below:**

{
  "player_name": "",
  "team": "",
  "birthdate": "",
  "position": "",
  "year": "",
  "conference": "",
  "archetype": "",
  "offense_summary": "",
  "defense_summary": "",
  "intangibles": "",
  "projection": ""
}

3. Only use information provided in RAW_SCOUTING_NOTES.
4. Do NOT hallucinate missing data.
5. Keep summaries concise but informative.
6. Use professional scouting terminology.
7. For archetype: choose a short, meaningful label
   (examples: "On-ball creator guard", "Rim-running energy big", "3&D wing",
    "Pick-and-pop big", "Two-way combo guard", etc.).
8. For projection: give a realistic 1–3 year outlook based ONLY on given notes.

---

RAW_SCOUTING_NOTES:
{input_text}

---

Now output ONLY the structured JSON scouting report. No explanations.
"""

---

## 🧩 Why This Teacher Prompt Is Designed This Way

A good teacher prompt must be:

### **1. Strict**
The student model learns *exact* formatting and structure.
Strict JSON = predictable downstream usage.

### **2. Complete**
The schema includes:
- biographical info
- archetype
- summaries
- projection

This allows future systems (like your similarity search and scouting dashboards)
to index and compare players consistently.

### **3. Deterministic**
Saying “No explanations. Output only JSON.”  
Prevents drifting into chatty output or additional text that breaks parsing.

### **4. Non-hallucinatory**
The teacher must only use provided info because
the student model will inherit behavior patterns.

### **5. Scouting-authentic**
Language, structure, tone — these must reflect real basketball scouting work
so internal/external users trust the output.

---

With this setup, we're ready to generate synthetic examples next.

---

In [11]:
from openai import OpenAI

client = OpenAI()

test_input = raw_examples[0].query  # Jamal Rivers example

prompt = TEACHER_PROMPT_TEMPLATE.format(input_text=test_input)

response = client.responses.create(
    model="gpt-4o-mini",  # or gpt-4o, gpt-4.1, llama3, etc.
    input=prompt
)

print(response.output_text)

ModuleNotFoundError: No module named 'openai'